In [1]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 50.4 MB 9.0 kB/s ta 0:00:0142   |█▎                              | 1.9 MB 5.2 MB/s eta 0:00:10     |█▊                              | 2.8 MB 5.2 MB/s eta 0:00:10     |██▎                             | 3.7 MB 5.2 MB/s eta 0:00:09     |███▏                            | 5.1 MB 5.2 MB/s eta 0:00:09     |████▏                           | 6.5 MB 5.2 MB/s eta 0:00:09     |████▌                           | 7.2 MB 5.2 MB/s eta 0:00:09     |█████                           | 7.8 MB 5.2 MB/s eta 0:00:09     |█████▍                          | 8.5 MB 5.2 MB/s eta 0:00:09     |███████▌                        | 11.9 MB 5.2 MB/s eta 0:00:08     |████████                        | 12.5 MB 5.2 MB/s eta 0:00:08     |████████▉                       | 13.9 MB 12.9 MB/s eta 0:00:03     |██████████▏                     | 15.9 MB 12.9 MB/s eta 0:00:03     |███████████                     | 17.3 

In [1]:
import cv2
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    if ds.Modality != 'DX':
        print('Modality not appropriate: ', ds.Modality, '\n')
        return None
    
    if ds.BodyPartExamined != 'CHEST':
        print('Body Part Examined not appropriate: ', ds.BodyPartExamined, '\n')
        return None
    
    if not (ds.PatientPosition == 'AP' or ds.PatientPosition == 'PA'):
        print('Patient Position not appropriate: ', ds.PatientPosition, '\n')
        return None
    
    print(ds.Modality)
    print(ds.BodyPartExamined)
    print(ds.PatientPosition)
    print('True: ', ds.StudyDescription)
    
    return img
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_size): 
    img = cv2.resize(img, img_size[1:-1])
    img = img / np.max(img)
    img = np.dstack((img, img, img))
    img = np.expand_dims(img, 0)
    
    return img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path) as json_file:
        model = keras.models.model_from_json(json_file.read())
        
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, threshold): 
    y_pred = model.predict(img)
    print(y_pred)
    y_pred = (y_pred >= threshold) * 1
    
    if y_pred == 1:
        prediction = 'Pneumonia'
    elif y_pred == 0:
        prediction = 'No Pneumonia'
    
    return prediction 

## Predictions

In [3]:
#check single img
img_ = check_dicom('test1.dcm')

Load file test1.dcm ...
DX
CHEST
PA
True:  No Finding


In [4]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'model.json'
weight_path = 'vgg16_best_dropout_4.hdf5'

IMG_SIZE = (1,224,224,3)
# img_mean = 0.5
# img_std = 0.5

model = load_model(model_path, weight_path)
threshold = 0.33

for i in test_dicoms:

    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img, IMG_SIZE)
    pred = predict_image(model, img_proc, threshold)
    print('Pred: ', pred, '\n')

Load file test1.dcm ...
DX
CHEST
PA
True:  No Finding
[[0.47920057]]
Pred:  Pneumonia 

Load file test2.dcm ...
DX
CHEST
AP
True:  Cardiomegaly
[[0.47035223]]
Pred:  Pneumonia 

Load file test3.dcm ...
DX
CHEST
AP
True:  Effusion
[[0.49844286]]
Pred:  Pneumonia 

Load file test4.dcm ...
Body Part Examined not appropriate:  RIBCAGE 

Load file test5.dcm ...
Modality not appropriate:  CT 

Load file test6.dcm ...
Patient Position not appropriate:  XX 



Along with the final prediction Pneumonia/No Pneumonia it might be beneficial to display the probability of the prediction to the radiologist as well.

## Check times

In [5]:
%time

img = check_dicom('test1.dcm')   
img_proc = preprocess_image(img, IMG_SIZE)
pred = predict_image(model, img_proc, threshold)
print('Pred: ', pred, '\n')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.34 µs
Load file test1.dcm ...
DX
CHEST
PA
True:  No Finding
[[0.47920057]]
Pred:  Pneumonia 



In [6]:
%time

img = check_dicom('test2.dcm')   
img_proc = preprocess_image(img, IMG_SIZE)
pred = predict_image(model, img_proc, threshold)
print('Pred: ', pred, '\n')

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.15 µs
Load file test2.dcm ...
DX
CHEST
AP
True:  Cardiomegaly
[[0.47035223]]
Pred:  Pneumonia 

